# SQL Final Project

<b> Project Description</b>

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.
We've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

<b>Task</b>

* Find the number of books released after January 1, 2000.
* Find the number of user reviews and the average rating for each book.
* Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
* Identify the author with the highest average book rating: look only at books with at least 50 ratings.
* Find the average number of text reviews among users who rated more than 50 books.

<b>Description of the data</b>

<b>books:</b>
    
Contains data on books:

`book_id`

`author_id`

`title`

`num_pages` — number of pages

`publication_date`

`publisher_id`


<b>authors:</b>

Contains data on `authors`:

`author_id`

`author`

<b>publishers:</b>

Contains data on publishers:

`publisher_id`

`publisher`

<b>ratings:</b>

Contains data on user ratings:

`rating_id`

`book_id`

`username` — the name of the user who rated the book

`rating`

<b>reviews:</b>

Contains data on customer reviews:

`review_id`

`book_id`

`username` — the name of the user who reviewed the book

`text` — the text of the review

# Table of Contents <a id='back'></a>

* [Introduction](#intro)
* [Stage 1. Data overview](#data_review)

* [Stage 2. Exploratory Data Analysis](#data_preprocessing)
    * [2.1 Number of books released after January 1, 2000](#header_style)
    * [2.2 Number of user reviews and the average rating for each book](#missing_values)
    * [2.3 Publisher that has released the greatest number of books with more than 50 pages](#duplicates)
    * [2.4 Author with the highest average book rating (books with at least 50 ratings)](#data_preprocessing_conclusions)
    * [2.5 Average number of text reviews among users who rated more than 50 books](#data_preprocessing_conclusions)
   
* [Stage 3. Conclusions](#end)

# Stage 1. Data Overview

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Connecting to the database
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [3]:
# a function to easily call a SQL variable
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [4]:
# preview of the books dataset
books = ''' SELECT 
            *
            FROM 
            books
        '''

#books= pd.io.sql.read_sql(query, con = engine, index_col = 'book_id')

queryResult(books).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
engine.table_names()

['orders',
 'visits',
 'advertisment_costs',
 'authors',
 'publishers',
 'reviews',
 'ratings',
 'books']

In [6]:
# preview of the authors dataset
authors = '''
           SELECT 
            *
           FROM
            authors
        '''
queryResult(authors).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
# preview of the publishers dataset
publishers = '''
           SELECT 
            *
           FROM
            publishers
        '''
queryResult(publishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
# preview of the ratings dataset
ratings = '''
           SELECT 
            *
           FROM
            ratings
        '''
queryResult(ratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
# preview of the reviews dataset
reviews = '''
           SELECT 
            *
           FROM
            reviews
        '''
queryResult(reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Stage 2. Exploratory Data Analysis

<b>Number of books released after January 1, 2000</b>

In [10]:
books_post_2000 = '''
                    SELECT 
                    COUNT (DISTINCT books.book_id) 
                    FROM 
                    books
                    WHERE 
                    publication_date > '2000-01-01'
                    '''
queryResult(books_post_2000)

,count
0,819


There are 819 books that were released after January 1st, 2000

<b>Number of user reviews and the average rating for each book</b>

In [ ]:
reviews_count = '''
                SELECT  
                books.title,
                COUNT (reviews.review_id),
                AVG(rating)
                
                FROM 
                books
                
                LEFT JOIN reviews ON 
                books.book_id = reviews.book_id
                LEFT JOIN ratings ON 
                books.book_id = ratings.book_id
                
                GROUP BY
                books.title
                
                ORDER BY
                count DESC
                
                '''

queryResult(reviews_count)

There are a total of 1000 books and we can see the number of reviews and average rating for each book

<b>Publisher that has released the greatest number of books with more than 50 pages</b>

In [ ]:

books_50_plus = '''
                SELECT
                publisher_id,
                COUNT (DISTINCT book_id)
                
                FROM
                books
                
                WHERE
                num_pages >= '50'
                
                GROUP BY
                publisher_id
                
                ORDER BY
                count desc
                '''

queryResult(books_50_plus)

In [ ]:
books_and_publishers = '''
                    SELECT 
                    publishers.publisher,
                    COUNT (DISTINCT book_id)
                    
                    
                    FROM
                    books
                    
                    INNER JOIN 
                    publishers
                    
                    ON books.publisher_id = publishers.publisher_id
                    
                    GROUP BY
                    publisher
                    
                    ORDER BY
                    COUNT (DISTINCT book_id) desc
                    LIMIT 1
                    
                    '''
                    

queryResult(books_and_publishers)

The publisher that has released the greatest number of books with more than 50 pages is Penguin Books

<b>Author with the highest average book rating (books with at least 50 ratings)</b>

In [ ]:
authors_ratings = '''
                    select *

                    FROM ratings
                    inner join books on ratings.book_id = books.book_id
                    inner join authors on books.book_id = authors.author_id
                    '''

queryResult(authors_ratings)


#SELECT column_name(s)
#FROM table1
#INNER JOIN table2
#ON table1.column_name = table2.column_name;

In [ ]:
#OVER (PARTITION BY book_id) as num_ratings

In [ ]:
high_ratings_author = '''
                        SELECT
                        author,
                        AVG(sub.avg_rating) AS high_avg_rating
                        FROM
                        (SELECT
                        author,
                        books.book_id,
                        AVG(rating) AS avg_rating
                        FROM 
                        authors
                        LEFT JOIN books ON books.author_id = authors.author_id
                        LEFT JOIN ratings ON ratings.book_id = books.book_id
                        GROUP BY
                        author,
                        books.book_id
                        HAVING
                        COUNT(rating) > 50) AS sub
                        GROUP BY
                        author
                        ORDER BY
                        high_avg_rating DESC
                        LIMIT
                        1  
                                         
                        '''
queryResult(high_ratings_author)

The author with the highest average book rating among books with at least 50 ratings is J.K. Rowling

<b>Average number of text reviews among users who rated more than 50 books</b>

In [ ]:
queryResult(reviews)

In [ ]:
avg_reviews = '''
                 SELECT
                AVG(count) AS avg_review_amount
                FROM
                (SELECT
                username,
                COUNT (DISTINCT review_id)
                FROM
                reviews
                WHERE
                username IN (SELECT
                            username
                            FROM
                            ratings
                            GROUP BY
                            username
                            HAVING
                            COUNT(DISTINCT book_id) > 50)
                GROUP BY
                username) s
                        
                        '''


queryResult(avg_reviews)

The average number of text reviews among users who rated more than 50 books is 24.3 reviews. 

# Stage 3. Conclusions

* There are 819 books that were released after January 1st, 2000

* There are a total of 1000 books and we can see the number of reviews and average rating for each book

* The publisher that has released the greatest number of books with more than 50 pages is Penguin Books

* The author with the highest average book rating among books with at least 50 ratings is J.K. Rowling

* The average number of text reviews among users who rated more than 50 books is 24.3 reviews. 

This information will be used to generate a value proposition for a new product.